In [1]:
def postprocessing_func(df, m) : 
    match_data = df[df['matchId'].values == m]
    num = match_data['maxPlace'].values[0]
    
    # empty  
    Maxplace = match_data.iloc[0]['maxPlace']
    Numgroups = match_data.iloc[0]['numGroups']
    t = Maxplace - Numgroups
    pivot = np.int(np.round((Maxplace - t)/(t+1)))
    indices = []
    for i in range(1, t+1) : 
        indices.append(i*pivot)
        
    match_data = match_data[['groupId', 'winPlacePercPred']].drop_duplicates()
    match_data = match_data.groupby('groupId')['winPlacePercPred'].mean().to_frame()
    match_data = match_data.sort_values(by = 'winPlacePercPred', ascending = False).reset_index()
    
    winPlace = np.linspace(0, 1, num = int(num), endpoint = True).tolist()
    winPlace = list(reversed(winPlace))
    remove_lst = [] 
    for idx in indices :
        remove_lst.append(winPlace[idx])
        
    for val in remove_lst : 
        if val in winPlace : 
            winPlace.remove(val)
        
    match_data['pred_winPlace'] = pd.Series(winPlace)
    
    pred_dict = match_data[['groupId', 'pred_winPlace']].set_index(['groupId']).to_dict()['pred_winPlace']
    rank_dictionary.update(pred_dict)
    
    return  